# Setup

In [ ]:
import tensorflow as tf
print(tf.__version__)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
tf.config.list_physical_devices('GPU') 

In [8]:
NUM_CLASSES = 8
BATCH_SIZE = 12
EPOCHS = 30
IMAGE_SIZE = 224

In [21]:
from tensorflow import keras

# Grab training and testing data
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "../input/car-brands/Car_Brand_Logos/Train",
    labels='inferred',
    label_mode='categorical',
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "../input/car-brands/Car_Brand_Logos/Test",
    labels='inferred',
    label_mode='categorical',
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
)

train_ds.class_names

Found 2499 files belonging to 8 classes.
Found 397 files belonging to 8 classes.


['hyundai',
 'lexus',
 'mazda',
 'mercedes',
 'opel',
 'skoda',
 'toyota',
 'volkswagen']

In [ ]:
import matplotlib.pyplot as plt

# Visualize data
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.axis("off")

In [ ]:
# Note to myself: 
# Tool to find most optimal hyperparameters with custom model
# source - https://github.com/keras-team/keras-tuner

# import keras_tuner

# def build_test_model(hp):
#     model = keras.Sequential()
#     model.add(layers.Rescaling(1.0 / 255))
#     model.add(layers.Conv2D(128, 3, strides=2, padding="same"))
#     model.add(layers.BatchNormalization())
#     model.add(layers.Dense(
#         # Tune number of units.
#         units=hp.Int("units", min_value=32, max_value=512, step=32),
#         # Tune the activation function to use.
#         activation=hp.Choice("activation", ["relu", "tanh"]),
#     ))
#     model.add(keras.layers.Dense(1, activation='relu'))
#     model.compile(loss='categorical_crossentropy')
#     return model

# tuner = keras_tuner.RandomSearch(
#     build_test_model,
#     objective='val_loss',
#     max_trials=10)

# tuner.search(train_ds, epochs=10, validation_data=val_ds)
# best_model = tuner.get_best_models()[0]
# best_model.build(input_shape=(image_size, image_size, 3))
# best_model.summary()

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetV2B0

# Using EfficientNet + Transfer Learning
# https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/#keras-implementation-of-efficientnet
img_augmentation = keras.Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
        layers.RandomBrightness(factor=0.1),
    ],
    name="img_augmentation",
)

def build_model(num_classes, image_size):
    inputs = layers.Input(shape=(image_size, image_size, 3))
    x = img_augmentation(inputs)
    model = EfficientNetV2B0(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam()
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

efficientnet_model = build_model(NUM_CLASSES, IMAGE_SIZE)
efficientnet_model.summary()

# Train

In [ ]:
# https://stackoverflow.com/questions/47843265/how-can-i-get-a-keras-models-history-after-loading-it-from-a-file-in-python
from keras.callbacks import CSVLogger

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),
    CSVLogger('training.log', separator=',', append=False)
]

In [ ]:
# Use GPU
with tf.device('/GPU:0'):
    history = efficientnet_model.fit(
        train_ds,
        epochs=EPOCHS,
        callbacks=callbacks,
        validation_data=val_ds,
    )

In [ ]:
efficientnet_model.save('efficientnet_model')

In [ ]:
!ls
model = keras.models.load_model('efficientnet_model')

# Predict

In [ ]:
import numpy as np

img = keras.utils.load_img(
    "../input/car-brands/Car_Brand_Logos/Test/volkswagen/images40.jpg", target_size=(IMAGE_SIZE, IMAGE_SIZE, 3)
)
img_array = keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
prediction_values = predictions.flatten()
print(np.column_stack((train_ds.class_names, prediction_values)))

max_val_index = max((v, i) for i, v in enumerate(predictions.flatten()))[1]

print(f"This car is a {train_ds.class_names[max_val_index]}")

# Analyze

In [ ]:
import matplotlib.pyplot as plt

def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

import pandas as pd
history = pd.read_csv('training.log', sep=',', engine='python')
plot_hist(history)

# Convert and Export

In [ ]:
# Download Model
!zip -r efficientnet_model.zip /kaggle/working/efficientnet_model

# from IPython.display import FileLink
# FileLink(r'efficientnet_model.zip') # gives you a download link to click on

In [16]:
import tensorflow as tf

# Optional: convert to tflite to run on mobile device
# https://www.tensorflow.org/lite/models/convert/convert_models
model = tf.saved_model.load('efficientnet_model')
concrete_func = model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
concrete_func.inputs[0].set_shape([None, IMAGE_SIZE, IMAGE_SIZE, 3])
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])

tflite_model = converter.convert()

In [20]:
# Add metadata to Tensorflow Lite model
# https://www.tensorflow.org/lite/models/convert/metadata
!pip install tflite_support_nightly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 MB 19.2 MB/s eta 0:00:0000:0100:01


In [ ]:
# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [30]:
# Create labelfile
with open("labelmap.txt", "w") as labelfile:
    for line in train_ds.class_names:
        labelfile.write(line + '\n')
    labelfile.close()

hyundai
lexus
mazda
mercedes
opel
skoda
toyota
volkswagen


In [33]:
!cat "labelmap.txt"

hyundai
lexus
mazda
mercedes
opel
skoda
toyota
volkswagen


In [37]:
# Add metadata to the tflite model
# https://www.tensorflow.org/lite/api_docs/python/tflite_support/metadata_writers/image_classifier
from tflite_support.metadata_writers import image_classifier
from tflite_support.metadata_writers import writer_utils
from tflite_support import metadata

ImageClassifierWriter = image_classifier.MetadataWriter
_MODEL_PATH = "model.tflite"
_LABEL_FILE = "labelmap.txt"
_SAVE_TO_PATH = "model_with_metadata.tflite"

writer = ImageClassifierWriter.create_for_inference(
    writer_utils.load_file(_MODEL_PATH), [127.5], [127.5], [_LABEL_FILE])
writer_utils.save_file(writer.populate(), _SAVE_TO_PATH)

# Verify the populated metadata and associated files.
displayer = metadata.MetadataDisplayer.with_model_file(_SAVE_TO_PATH)
print("Metadata populated:")
print(displayer.get_metadata_json())
print("Associated file(s) populated:")
print(displayer.get_packed_associated_file_list())

Metadata populated:
{
  "name": "ImageClassifier",
  "description": "Identify the most prominent object in the image from a known set of categories.",
  "subgraph_metadata": [
    {
      "input_tensor_metadata": [
        {
          "name": "image",
          "description": "Input image to be classified.",
          "content": {
            "content_properties_type": "ImageProperties",
            "content_properties": {
              "color_space": "RGB"
            }
          },
          "process_units": [
            {
              "options_type": "NormalizationOptions",
              "options": {
                "mean": [
                  127.5
                ],
                "std": [
                  127.5
                ]
              }
            }
          ],
          "stats": {
            "max": [
              1.0
            ],
            "min": [
              -1.0
            ]
          }
        }
      ],
      "output_tensor_metadata": [
        {
    

In [38]:
# https://stackoverflow.com/questions/24437661/retrieving-files-from-remote-ipython-notebook-server
from IPython.display import FileLink

FileLink(r'model_with_metadata.tflite')

/kaggle/working/model_with_metadata.tflite